In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
!pip install pandas

In [2]:
!pip install mysql-connector-python

In [3]:
import sys
import traceback
import logging

sys.path.append('/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages')

In [4]:
from bs4 import BeautifulSoup
import requests
import os
import csv
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup as soup

print("Succssfully imported libraries")

Succssfully imported libraries


In [5]:
url = 'https://www.worldometers.info/coronavirus/'
worldo_meter_response = requests.get(url, allow_redirects = True)
worldo_meter_response.status_code

200

In [6]:
soup_response = soup(worldo_meter_response.text, "html.parser")

In [7]:
sections = soup_response("div", id = "maincounter-wrap")

In [8]:
total_cases = sections[0].find('span').text.replace(" ", "").replace(",", "")
total_deaths = sections[1].find('span').text.replace(" ", "").replace(",", "")
total_recovered = sections[2].find('span').text.replace(" ", "").replace(",", "")

In [9]:
print(f'Total cases : {total_cases}')
print(f'Total deaths : {total_deaths}')
print(f'Total recovered : {total_recovered}')

Total cases : 513501075
Total deaths : 6261272
Total recovered : 467505073


Country specific covid data

In [10]:
import re
import json
import pprint

In [11]:
def get_countries():
  url = 'https://restcountries.com/v3.1/all'
  countries = requests.get(url, allow_redirects = True)
  raw_countries = countries.text
  return raw_countries

In [12]:
def process_raw_countries(raw_countries):
  countries = []
  countries_json = json.loads(raw_countries)
  for country in countries_json:
    countries.append(country['name']['common'])
  return countries

In [13]:
import mysql.connector
def get_connection():
    try:
        cnx = mysql.connector.connect(user='safetravels_user', password='safetravels_pwd',
                                  host='localhost',
                                  database='safetravels')
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
    return cnx

def close_connection(cnx):
    cnx.close()

In [14]:
def add_countries():
    cnx = None
    try:
        raw_countries = get_countries()
        countries_list = process_raw_countries(raw_countries = raw_countries)
        cnx = get_connection()
        cursor = cnx.cursor()
        add_country = "insert into Countries (CountryName, AlternateName) values (%s, %s);"
        for country in countries_list:
            data_country = (country, country)
            cursor.execute(add_country, data_country)
            country_id = cursor.lastrowid
            cnx.commit()
    except Exception as e:
        logging.error(traceback.format_exc())
    finally:
        if cnx != None:
            close_connection(cnx)

In [15]:
error_countries = set()

In [16]:
add_countries()

In [17]:
def get_cdc_advises(country):
    try:
        url = f'https://wwwnc.cdc.gov/travel/notices/covid-4/coronavirus-{country}'
        cdc_response = requests.get(url, allow_redirects = True)
        cdc_object = soup(cdc_response.text, "html.parser")
        assert cdc_object != None, f"Error occured during parsing {get_cdc_advises.__name__}"
        notice = cdc_object.find("div", class_ = re.compile("notice-typename.*"))
        #notice = entry_details.find('div') #, class_ = "notice-typename-covid-4 p-2")
        if notice == None:
            print()
            print(f'Notice cannot be empty. Error country : {country}')
            error_countries.add(country)
            return None, None
        notice = notice.text
        advices = cdc_object.find(class_ = 'card-body')
        list_items = advices.findAll('li')
        return_advices = []
        for a in list_items:
          return_advices.append(a.text)
        notice = notice.split(':')[0].split(' ')[1]
        return notice, return_advices
    except Exception as e:
        error_countries.add(country)
        print('Exception occured with country ' + country)
        logging.error(traceback.format_exc())
        return None, None

def get_countries():
    country_list = []
    refined_country_list = []
    cnx = get_connection()
    cursor = cnx.cursor()
    countries_query = "select CountryName from Countries;"
    cursor.execute(countries_query)
    for (CountryName) in cursor:
        country = CountryName[0]
        temp_countries = country.split(' ')
        temp_countries = [a.lower() for a in temp_countries]
        refined_country = '-'.join(temp_countries)
        country_list.append(CountryName[0])
        refined_country_list.append(refined_country)
    return country_list, refined_country_list

In [18]:
def get_country_id(country):
    sql = f'select CountryID from Countries where CountryName = %s'
    cnx = get_connection()
    cursor = cnx.cursor()
    result = cursor.execute(sql, (country,), multi=True)
    countryID = None
    for r in result:
        rows = r.fetchall()
        countryID = rows[0][0]
        break
    return countryID

In [33]:
def fill_advisories(level_name, country_id, sub_advices):
    cnx = None
    try:
        add_advice_sql = "insert into Advisory (LevelName, CountryID) values (%s, %s);"
        cnx = get_connection()
        cursor = cnx.cursor()
        cursor.execute(add_advice_sql, (level_name, country_id))
        advice_id = cursor.lastrowid
        for sub_advice in sub_advices:
            add_subadvice_sql = "insert into SubAdvisory (SubAdvisoryText, AdvisoryID) values (%s, %s)"
            cursor.execute(add_subadvice_sql, (sub_advice, advice_id))
        cnx.commit()
        print('advice id', advice_id)
    except Exception as e:
        logging.error(traceback.format_exc())
    finally:
        if cnx != None:
            close_connection(cnx)

In [39]:
advisory_failed_countries = set()

In [44]:
def populate_advisories():
    country_list, refined_country_list = get_countries()
    for country, refined_country in zip(country_list, refined_country_list):
        level, sub_advices = get_cdc_advises(refined_country)
        country_id = get_country_id(country)
        if level == None:
            print('Level cannot be None')
            print('Country', country)
            advisory_failed_countries.add(country)
            continue
        if country_id == None:
            advisory_failed_countries.add(country)
            print(f"Failed to populate advisory for {country}")
            continue
        fill_advisories(level, country_id, sub_advices)

In [45]:
populate_advisories()

advice id 12

Notice cannot be empty. Error country : åland-islands
Level cannot be None
Country Åland Islands
advice id 13
advice id 14

Notice cannot be empty. Error country : american-samoa
Level cannot be None
Country American Samoa
advice id 15
advice id 16
advice id 17
advice id 18
advice id 19
advice id 20
advice id 21
advice id 22
advice id 23
advice id 24
advice id 25

Notice cannot be empty. Error country : bahamas
Level cannot be None
Country Bahamas
advice id 26
advice id 27
advice id 28
advice id 29
advice id 30
advice id 31
advice id 32
advice id 33
advice id 34
advice id 35
advice id 36
advice id 37

Notice cannot be empty. Error country : bouvet-island
Level cannot be None
Country Bouvet Island
advice id 38


ERROR:root:Traceback (most recent call last):
  File "/var/folders/_5/4xl0ylmn6vz97z82pd3c33g00000gn/T/ipykernel_767/41250765.py", line 20, in get_cdc_advises
    notice = notice.split(':')[0].split(' ')[1]
IndexError: list index out of range



Exception occured with country british-indian-ocean-territory
Level cannot be None
Country British Indian Ocean Territory
advice id 39
advice id 40
advice id 41
advice id 42
advice id 43
advice id 44
advice id 45
advice id 46
advice id 47

Notice cannot be empty. Error country : caribbean-netherlands
Level cannot be None
Country Caribbean Netherlands
advice id 48
advice id 49
advice id 50
advice id 51
advice id 52
advice id 53

Notice cannot be empty. Error country : cocos-(keeling)-islands
Level cannot be None
Country Cocos (Keeling) Islands
advice id 54
advice id 55
advice id 56
advice id 57
advice id 58
advice id 59

Notice cannot be empty. Error country : curaçao
Level cannot be None
Country Curaçao
advice id 60

Notice cannot be empty. Error country : czechia
Level cannot be None
Country Czechia
advice id 61
advice id 62
advice id 63
advice id 64

Notice cannot be empty. Error country : dr-congo
Level cannot be None
Country DR Congo
advice id 65
advice id 66
advice id 67
advice id

ERROR:root:Traceback (most recent call last):
  File "/var/folders/_5/4xl0ylmn6vz97z82pd3c33g00000gn/T/ipykernel_767/41250765.py", line 20, in get_cdc_advises
    notice = notice.split(':')[0].split(' ')[1]
IndexError: list index out of range



Exception occured with country western-sahara
Level cannot be None
Country Western Sahara
advice id 223
advice id 224
advice id 225


TODO : Fix infromation for these countries

In [46]:
advisory_failed_countries

{'American Samoa',
 'Bahamas',
 'Bouvet Island',
 'British Indian Ocean Territory',
 'Caribbean Netherlands',
 'Cocos (Keeling) Islands',
 'Curaçao',
 'Czechia',
 'DR Congo',
 'French Southern and Antarctic Lands',
 'Gambia',
 'Guam',
 'Heard Island and McDonald Islands',
 'Marshall Islands',
 'Micronesia',
 'Myanmar',
 'Northern Mariana Islands',
 'Palau',
 'Palestine',
 'Puerto Rico',
 'Republic of the Congo',
 'Réunion',
 'Saint Barthélemy',
 'Saint Helena, Ascension and Tristan da Cunha',
 'Saint Kitts and Nevis',
 'South Georgia',
 'Svalbard and Jan Mayen',
 'São Tomé and Príncipe',
 'Turks and Caicos Islands',
 'United States',
 'United States Minor Outlying Islands',
 'United States Virgin Islands',
 'Vatican City',
 'Wallis and Futuna',
 'Western Sahara',
 'Åland Islands'}